In [15]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import csv
import numpy as np

In [27]:
ano = '2020'

r = requests.get('http://www.jogosdoguarani.com/'+ano+'/index.html')

soup = BeautifulSoup(r.text, 'html.parser')


#list de jogos 
lst_jogos = []

try:
    
    #extracao 
    estadio = 0
    localidade = ''
    campeonato=0
    for link in soup.find_all('a'):
        rjogo = requests.get('http://www.jogosdoguarani.com/'+ano+'/'+link.get('href'))
        soupjogo = BeautifulSoup(rjogo.text, 'html.parser')

        #Local do jogo
        if (soupjogo.title.string[0:7]=='GUARANI'):
            local = 'C'
        else:
            local = 'F'
        #Adversario    
        if (local=='C'):
            #Adversario 
            adversario = soupjogo.title.string.split(sep=" ", maxsplit=4)[4]
            #gols adversario
            gol_adversario = soupjogo.title.string.split(sep=" ", maxsplit=4)[3]
            #gols guarani
            gol_guarani = soupjogo.title.string.split(sep=" ", maxsplit=4)[1]
        else:
            #Adversario 
            adversario = soupjogo.title.string.rsplit(sep=" ", maxsplit=4)[0]
            #gols adversario
            gol_adversario = soupjogo.title.string.rsplit(sep=" ", maxsplit=4)[1]
            #gols guarani
            gol_guarani = soupjogo.title.string.rsplit(sep=" ", maxsplit=4)[3]

        if (gol_guarani > gol_adversario):
            result = 'V'
        elif (gol_guarani < gol_adversario):
            result = 'D'
        else:
            result = 'E'


        #data
        infos = soupjogo.text.splitlines()
        for info in infos:

            if ("Data" in info):
                data = info.replace("Data: ", "").replace(" (sábado)", "").replace("/", "-")
            if ("Horário" in info):
                horario = (info.replace("Horário: ", "").replace("h", ":")+("00"))[:5]  
            if ("Tempo" in info):
                clima = info.replace("Tempo:", "").lstrip()
            if (estadio==1 and info!=''):
                localidade = info.lstrip()
                estadio = 0
            if ("Local" in info and estadio==0):
                estadio = 1
            if (campeonato==1 and info!=''):
                competicao = info.lstrip()
                campeonato = 0
            if ("Campeonato" in info and campeonato==0):
                campeonato = 1

        #adiciona elementos lista 
        lst_jogos.append([link.get('href').replace(".html",""), soupjogo.title.string, result, local, adversario, gol_guarani, gol_adversario, data, horario, clima, localidade, competicao, 'http://www.jogosdoguarani.com/'+ano+'/'+link.get('href')])
        
        print("OK - ITEM ADICIONADO " + soupjogo.title.string)
        #print(link.get('href').replace(".html","") + 
        #   ("|") + soupjogo.title.string + 
        #   ("|") + result +
        #   ("|") + local + 
        #   ("|") + adversario+
        #   ("|") + gol_guarani+
        #   ("|") + gol_adversario+
        #   ("|") + data+
        #   ("|") + horario+
        #   ("|") +   clima+
        #   ("|") + localidade+
        #   ("|") +  competicao+
        #   ("|") + 'http://www.jogosdoguarani.com/2021/'+link.get('href')
        #   )
        #
    #fecha o arquivo após finalizar sem erro     
except:
    print("")
    print("=========== AÇÃO FINALIZADA  0 ===========")

#criação do dataframe

df_jogos = pd.DataFrame(lst_jogos, columns=['id', 'jogo', 'resultado', 'local', 'adversario', 'gol_guarani', 'gol_adversario', 'data', 'horario', 'clima', 'localidade', 'campeonato', 'link'])
df_jogos.to_csv('/home/marcus-pinheiro/project/python/EGE/jogos_'+ano+'.csv', sep=';')



OK - ITEM ADICIONADO Internacional (Limeira) 0 X 4 GUARANI
OK - ITEM ADICIONADO GUARANI 1 X 2 Santos
OK - ITEM ADICIONADO Mirassol 1 X 1 GUARANI
OK - ITEM ADICIONADO GUARANI 2 X 1 Santo André
OK - ITEM ADICIONADO Oeste 1 X 1 GUARANI
OK - ITEM ADICIONADO GUARANI 1 X 1 Novorizontino
OK - ITEM ADICIONADO Palmeiras 1 X 0 GUARANI
OK - ITEM ADICIONADO GUARANI 0 X 0 Água Santa
OK - ITEM ADICIONADO Ituano 0 X 2 GUARANI
OK - ITEM ADICIONADO GUARANI 3 X 2 AAPP
OK - ITEM ADICIONADO Botafogo (Rib. Preto) 2 X 0 GUARANI
OK - ITEM ADICIONADO GUARANI 1 X 3 S. Paulo
OK - ITEM ADICIONADO GUARANI 1 X 1 Ituano
OK - ITEM ADICIONADO Internacional (Limeira) 1 X 2 GUARANI
OK - ITEM ADICIONADO Red Bull Bragantino 1 X 0 GUARANI
OK - ITEM ADICIONADO CSA 1 X 0 GUARANI
OK - ITEM ADICIONADO GUARANI 2 X 3 Cruzeiro
OK - ITEM ADICIONADO Botafogo (Rib. Preto) 0 X 1 GUARANI
OK - ITEM ADICIONADO GUARANI 1 X 2 Paraná
OK - ITEM ADICIONADO Chapecoense 2 X 0 GUARANI
OK - ITEM ADICIONADO GUARANI 1 X 2 Náutico
OK - ITEM ADICIO

In [28]:
df_teste =  pd.read_csv('/home/marcus-pinheiro/project/python/EGE/jogos_'+ano+'.csv', sep=';')


df_teste

,Unnamed: 0,id,jogo,resultado,local,adversario,gol_guarani,gol_adversario,data,horario,clima,localidade,campeonato,link
0,0,0120,Internacional (Limeira) 0 X 4 GUARANI,V,F,Internacional (Limeira),4,0,22-01-2020 (quarta-feira),19:15,"nublado com chuva no final, 29-24°C",Major José Levy Sobrinho (Limeirão) - Limeira/SP,Paulista 2020 - 1ª Fase - 1ª Rodada,http://www.jogosdoguarani.com/2020/0120.html
1,1,0220,GUARANI 1 X 2 Santos,D,C,Santos,1,2,27-01-2020 (segunda-feira),20:00,NaN,Brinco de Ouro da Princesa - Campinas/SP,Paulista 2020 - 1ª Fase - 2ª Rodada,http://www.jogosdoguarani.com/2020/0220.html
2,2,0320,Mirassol 1 X 1 GUARANI,E,F,Mirassol,1,1,30-01-2020,21:30,NaN,José Maria de Campos Maia - Mirassol/SP,Paulista 2020 - 1ª Fase - 3ª Rodada,http://www.jogosdoguarani.com/2020/0320.html
3,3,0420,GUARANI 2 X 1 Santo André,V,C,Santo André,2,1,02-02-2020 (domingo),19:00,NaN,Brinco de Ouro da Princesa - Campinas/SP,Paulista 2020 - 1ª Fase - 4ª Rodada,http://www.jogosdoguarani.com/2020/0420.html
4,4,0520,Oeste 1 X 1 GUARANI,E,F,Oeste,1,1,07-02-2020 (sexta-feira),19:15,NaN,Orlando Baptista Novelli (Arena Barueri) - Bar...,Paulista 2020 - 1ª Fase - 5ª Rodada,http://www.jogosdoguarani.com/2020/0520.html
5,5,0620,GUARANI 1 X 1 Novorizontino,E,C,Novorizontino,1,1,15-02-2020,16:00,NaN,Brinco de Ouro da Princesa - Campinas/SP,Paulista 2020 - 1ª Fase - 6ª Rodada,http://www.jogosdoguarani.com/2020/0620.html
6,6,0720,Palmeiras 1 X 0 GUARANI,D,F,Palmeiras,0,1,20-02-2020,21:30,NaN,Arena Palestra Itália (Allianz Parque) - São P...,Paulista 2020 - 1ª Fase - 7ª Rodada,http://www.jogosdoguarani.com/2020/0720.html
7,7,0820,GUARANI 0 X 0 Água Santa,E,C,Água Santa,0,0,28-02-2020 (sexta-feira),19:15,NaN,Brinco de Ouro da Princesa - Campinas/SP,Paulista 2020 - 1ª Fase - 8ª Rodada,http://www.jogosdoguarani.com/2020/0820.html
8,8,0920,Ituano 0 X 2 GUARANI,V,F,Ituano,2,0,06-03-2020 (sexta-feira),21:30,NaN,Dr. Luiz Gonzaga Novelli Júnior - Itu/SP,Paulista 2020 - 1ª Fase - 9ª Rodada,http://www.jogosdoguarani.com/2020/0920.html
9,9,1020,GUARANI 3 X 2 AAPP,V,C,AAPP,3,2,16-03-2020 (segunda-feira),20:00,NaN,Brinco de Ouro da Princesa - Campinas/SP,Paulista 2020 - 1ª Fase - 10ª Rodada,http://www.jogosdoguarani.com/2020/1020.html
